1. torch summary
2. tensorboard

In [3]:
from torchsummary import summary

In [2]:
!git clone https://github.com/fastai/course-v3
%cd /content/course-v3/nbs/dl2
from exp.nb_08 import *
path = datasets.untar_data(datasets.URLs.IMAGENETTE_160)

tfms = [make_rgb, ResizeFixed(128), to_byte_tensor, to_float_tensor]
bs = 128
il = ImageList.from_files(path, tfms=tfms)
sd = SplitData.split_by_func(il, partial(grandparent_splitter, valid_name='val'))
ll = label_by_func(sd, parent_labeler, proc_y=CategoryProcessor())
data = ll.to_databunch(bs, c_in=3, c_out=10, num_workers=2)

nfs = [32, 64, 128, 256]

cbfs = [partial(AvgStatsCallback, accuracy),
        CudaCallback,
        partial(BatchTransformXCallback, norm_imagenette)]

learn, run= get_learn_run(nfs, data, 0.4, conv_layer, cbs=cbfs)        

Cloning into 'course-v3'...
remote: Enumerating objects: 5893, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 5893 (delta 0), reused 2 (delta 0), pack-reused 5890
Receiving objects: 100% (5893/5893), 263.03 MiB | 40.17 MiB/s, done.
Resolving deltas: 100% (3249/3249), done.
/content/course-v3/nbs/dl2


In [12]:
next(iter(learn.data.train_dl))[1].shape

torch.Size([128])

In [17]:
summary(learn.model.cuda(), input_size = learn.data.train_ds[0][0].shape, batch_size=128)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1        [128, 16, 128, 128]             432
       GeneralRelu-2        [128, 16, 128, 128]               0
       BatchNorm2d-3        [128, 16, 128, 128]              32
            Conv2d-4          [128, 32, 64, 64]           4,608
       GeneralRelu-5          [128, 32, 64, 64]               0
       BatchNorm2d-6          [128, 32, 64, 64]              64
            Conv2d-7          [128, 64, 32, 32]          18,432
       GeneralRelu-8          [128, 64, 32, 32]               0
       BatchNorm2d-9          [128, 64, 32, 32]             128
           Conv2d-10          [128, 32, 16, 16]          18,432
      GeneralRelu-11          [128, 32, 16, 16]               0
      BatchNorm2d-12          [128, 32, 16, 16]              64
           Conv2d-13            [128, 64, 8, 8]          18,432
      GeneralRelu-14            [128, 6

In [22]:
import inspect; inspect.getfullargspec(summary)

FullArgSpec(args=['model', 'input_size', 'batch_size', 'device'], varargs=None, varkw=None, defaults=(-1, 'cuda'), kwonlyargs=[], kwonlydefaults=None, annotations={})